# Supersymmetric harmonic oscillator

As a warmup, lets go through the steps to get from the Hamiltonian of this system to a quantum circuit that lets us evolve states according to the time evolution operator.

The superpotential is 
\begin{equation}
    W(\hat{q}) = \frac{1}{2}m \hat{q}^2
\end{equation}
which leads to the following Hamiltonian,
\begin{equation}
    H = \frac{1}{2}\left(H_B+H_F\right), \quad 
        H_B=\hat{p}^2+m^2\hat{q}^2, \quad H_F=m\left[\hat{b}^{\dagger},\hat{b}\right].
\end{equation}
This contains two terms, a bosonic piece and a fermionic piece.  For the bosonic part, we could work in the position or momentum basis, but this is just the Hamiltonian for a harmonic oscillator, so lets convert to the number basis.  


In [22]:
import sys
sys.path.append('..')
from src.sympy_utilities import *
from src.BinaryEncodings import *

import sympy as sp

h_b = 0.5*(p*p + m*m*q*q)

print('Original H = ' + str(h_b))
h_b = sp.expand(h_b.subs(qp_to_ada))
h_f = -0.5*m#[bdag,b]
print('Using creation/annihlation operators\nH_B = ' + str(h_b))
print('H_F = ' + str(h_f))

Original H = 0.5*(m**2*q**2 + p**2)
Using creation/annihlation operators
H_B = 0.5*m*a*ad + 0.5*m*ad*a
H_F = -0.5*m


And we know the matrix elements of $A$ and $A^{\dagger}$, they are defined in HamiltonianTerms.  We of course must impose a cutoff at this point, truncating the allowed excitations of the harmonic oscillator

In [23]:
from src.BinaryEncodings import *

hamHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                     2, standard_encode)
print('done')
print()
print(hamHO.bmatrix)
print(np.kron(np.eye(2),hamHO.bmatrix))
print(hamHO.bosonPauliStrings)
print()
print(hamHO.fermionic)
print(hamHO.fmatrix)
print(sp.simplify(hamHO.fermionPauliStrings))
print()

print(hamHO.pauliStrings)
print(hamHO.hamMatrix)

done

[[0.500000000000000 0]
 [0 1.50000000000000]]
[[0.500000000000000 0 0 0]
 [0 1.50000000000000 0 0]
 [0 0 0.500000000000000 0]
 [0 0 0 1.50000000000000]]
1.0*I^0 - 0.5*Z^0

-0.500000000000000
[[0.500000000000000 0 0 0]
 [0 0.500000000000000 0 0]
 [0 0 -0.500000000000000 0]
 [0 0 0 -0.500000000000000]]
0.5*I^0*Z^1

1.0*I^0*I^1 + 0.5*I^0*Z^1 - 0.5*I^1*Z^0
[[ 1.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.11022302e-16+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  2.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   1.00000000e+00+0.j]]


In [26]:
from src.BinaryEncodings import *

hamHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                     3, standard_encode)
print('done')
print()
print(hamHO.bmatrix)
print(np.kron(np.eye(2),hamHO.bmatrix))
print(hamHO.bosonPauliStrings)
print()
print(hamHO.fermionic)
print(hamHO.fmatrix)
print(sp.simplify(hamHO.fermionPauliStrings))
print()

print(hamHO.pauliStrings)
print(hamHO.hamMatrix)

done

[[0.500000000000000 0 0]
 [0 1.50000000000000 0]
 [0 0 2.50000000000000]]
[[0.500000000000000 0 0 0 0 0]
 [0 1.50000000000000 0 0 0 0]
 [0 0 2.50000000000000 0 0 0]
 [0 0 0 0.500000000000000 0 0]
 [0 0 0 0 1.50000000000000 0]
 [0 0 0 0 0 2.50000000000000]]
1.125*I^0*I^1 - 0.125*I^0*Z^1 + 0.375*I^1*Z^0 - 0.875*Z^0*Z^1

-0.500000000000000
[[0.500000000000000 0 0 0 0 0]
 [0 0.500000000000000 0 0 0 0]
 [0 0 0.500000000000000 0 0 0]
 [0 0 0 -0.500000000000000 0 0]
 [0 0 0 0 -0.500000000000000 0]
 [0 0 0 0 0 -0.500000000000000]]
0.25*I^0*I^1*Z^2 + 0.25*I^0*Z^1*Z^2 + 0.125*I^1*I^2*Z^0 + 0.125*I^1*Z^0*Z^2 - 0.125*I^2*Z^0*Z^1 - 0.125*Z^0*Z^1*Z^2

1.125*I^0*I^1*I^2 + 0.25*I^0*I^1*Z^2 - 0.125*I^0*I^2*Z^1 + 0.25*I^0*Z^1*Z^2 + 0.5*I^1*I^2*Z^0 + 0.125*I^1*Z^0*Z^2 - 1.0*I^2*Z^0*Z^1 - 0.125*Z^0*Z^1*Z^2
[[ 1.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  5.5511

# Supersymmetric Anharmonic Oscillator

The superpotential is 
\begin{equation}
    W(\hat{q}) = \frac{1}{2}m\hat{q}^2 + \frac{1}{4}g\hat{q}^4
\end{equation}

The full Hamiltonian is
\begin{equation}
    H=\frac{1}{2}\left[\hat{p}^2 + m^2\hat{q}^2 + 2mg\hat{q}^4 + g^2\hat{q}^6 - (m+3g\hat{q}^2)\left[b^{\dagger},b\right]\right]
\end{equation}

In [3]:
import sys
sys.path.append('..')
from src.sympy_utilities import *
from src.BinaryEncodings import *

import sympy as sp

h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = -0.5*(m+3*g*q*q)#[bdag,b]


n=2
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                    n, standard_encode)

In [4]:
print('Using creation/annihlation operators\nH_B = ' + str(hamAHO.harmonic))
print()
print('H_F = ' + str(hamAHO.fermionic))

Using creation/annihlation operators
H_B = 0.5*a*ad + 0.25*a*ad*a*ad + 0.0625*a*ad*a*ad*a*ad + 0.0625*a*ad*a*ad*a**2 + 0.0625*a*ad*a*ad**2*a + 0.0625*a*ad*a*ad**3 + 0.25*a*ad*a**2 + 0.0625*a*ad*a**2*ad*a + 0.0625*a*ad*a**2*ad**2 + 0.0625*a*ad*a**3*ad + 0.0625*a*ad*a**4 + 0.25*a*ad**2*a + 0.0625*a*ad**2*a*ad*a + 0.0625*a*ad**2*a*ad**2 + 0.0625*a*ad**2*a**2*ad + 0.0625*a*ad**2*a**3 + 0.25*a*ad**3 + 0.0625*a*ad**3*a*ad + 0.0625*a*ad**3*a**2 + 0.0625*a*ad**4*a + 0.0625*a*ad**5 + 0.25*a**2*ad*a + 0.0625*a**2*ad*a*ad*a + 0.0625*a**2*ad*a*ad**2 + 0.0625*a**2*ad*a**2*ad + 0.0625*a**2*ad*a**3 + 0.25*a**2*ad**2 + 0.0625*a**2*ad**2*a*ad + 0.0625*a**2*ad**2*a**2 + 0.0625*a**2*ad**3*a + 0.0625*a**2*ad**4 + 0.25*a**3*ad + 0.0625*a**3*ad*a*ad + 0.0625*a**3*ad*a**2 + 0.0625*a**3*ad**2*a + 0.0625*a**3*ad**3 + 0.25*a**4 + 0.0625*a**4*ad*a + 0.0625*a**4*ad**2 + 0.0625*a**5*ad + 0.0625*a**6 + 0.5*ad*a + 0.25*ad*a*ad*a + 0.0625*ad*a*ad*a*ad*a + 0.0625*ad*a*ad*a*ad**2 + 0.0625*ad*a*ad*a**2*ad + 0.0625*ad*a*

In [5]:
print(hamAHO.fermionPauliStrings)

2.0*I^0*Z^1 - 0.75*Z^0*Z^1


In [6]:
print(np.kron(np.eye(2),hamAHO.bmatrix))
print(hamAHO.fmatrix)

[[2.18750000000000 0 0 0]
 [0 11.8125000000000 0 0]
 [0 0 2.18750000000000 0]
 [0 0 0 11.8125000000000]]
[[1.25000000000000 0 0 0]
 [0 2.75000000000000 0 0]
 [0 0 -1.25000000000000 0]
 [0 0 0 -2.75000000000000]]


In [7]:
h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = -0.5*(m+3*g*q*q)#[bdag,b]


n=2
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:0},
                    n, standard_encode)

In [8]:
hamAHO.pauliStrings

1.0*I^0*I^1 + 0.5*I^0*Z^1 - 0.5*I^1*Z^0

In [9]:
h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = 0.5*(m+3*g*q*q)#[bdag,b]


n=4
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                    n, standard_encode)

In [10]:
hamAHO.pauliStrings

32.75*I^0*I^1*I^2 - 3.5*I^0*I^1*Z^2 + 14.1486483908533*I^0*I^2*X^1 - 25.75*I^0*I^2*Z^1 - 1.4488887394336*I^0*X^1*Z^2 + 1.5*I^0*Z^1*Z^2 - 13.8125*I^1*I^2*Z^0 + 0.75*I^1*Z^0*Z^2 - 8.04985240311929*I^2*X^1*Z^0 + 9.0*I^2*Z^0*Z^1 + 0.388228567653781*X^1*Z^0*Z^2

In [11]:
hamAHO.hamMatrix

array([[ 0.9375    +0.j,  0.        +0.j,  5.03813582+0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  3.4375    +0.j,  0.        +0.j,
         7.15945616+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 5.03813582+0.j,  0.        +0.j, 31.4375    +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  7.15945616+0.j,  0.        +0.j,
        39.9375    +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  9.0625    +0.j,  0.        +0.j,
        20.36138349+0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j, 14.5625    +0.j,
         0.        +0.j, 24.0356181 +0.j],
       [ 0.        +0.j,  0.      

In [12]:
import sys
sys.path.append('..')
from src.sympy_utilities import *
convert_to_matrix(0.75*a*a,4,8)

array([[0, 0, 1.06066017177982, 0],
       [0, 0, 0, 1.83711730708738],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=object)

In [13]:
convert_to_matrix(0.75*a*adag,4,8)

array([[0.750000000000000, 0, 0, 0],
       [0, 1.50000000000000, 0, 0],
       [0, 0, 2.25000000000000, 0],
       [0, 0, 0, 3.00000000000000]], dtype=object)

In [14]:
expr = a*adag
expr2 = adag*a

In [15]:
expr

a*ad

In [16]:
convert_to_matrix(expr,2,2)

array([[1.00000000000000, 0],
       [0, 2.00000000000000]], dtype=object)

In [17]:
convert_to_matrix(expr2,2,2)

array([[0, 0],
       [0, 1.00000000000000]], dtype=object)

In [18]:
convert_term_to_matrix(expr,4)

array([[1.00000000000000, 0, 0, 0],
       [0, 2.00000000000000, 0, 0],
       [0, 0, 3.00000000000000, 0],
       [0, 0, 0, 0]], dtype=object)

In [19]:
convert_term_to_matrix(expr2,4)

array([[0, 0, 0, 0],
       [0, 1.00000000000000, 0, 0],
       [0, 0, 2.00000000000000, 0],
       [0, 0, 0, 3.00000000000000]], dtype=object)